# pytorch实现自动求导autograd
torch.autograd就是为方便用户使用，而专门开发的一套自动求导引擎，它能够根据输入和前向传播过程自动构建计算图，并执行反向传播

计算图(Computation Graph)是现代深度学习框架如PyTorch和TensorFlow等的核心，其为高效自动求导算法——反向传播(Back Propogation)提供了理论支持，了解计算图在实际写程序过程中会有极大的帮助。计算图的相关内容在吴恩达深度学习课程和邱希鹏《神经网络与深度学习》中，均有讲到，可以看我之前的[博客](https://xiuzhedorothy.gitee.io/2020/01/19/qian-kui-shen-jing-wang-luo-xiao-jie/ "博客")😀
![](https://s2.ax1x.com/2020/01/19/1CNLo6.png)

## requires_grad
autograd中的核心数据结构是Variable。从v0.4版本起，Variable和Tensor合并。可以认为**需要求导的tensor**(requires_grad)即Variable。

Variable提供了大部分tensor支持的函数，但不支持部分inplace函数，因为它们会修改tensor自身，而在反向传播中，variable需要缓存原来的tensor来计算反向传播梯度。计算各个Variable的梯度，调用它们根节点Variable的`backward`方法即可，autograd会自动沿着计算图反向传播，计算每一个叶子节点的梯度(下来会举例解释)

In [1]:
#pre
import torch as t

`variable.backward(gradient=None, retain_graph=None, create_graph=None)`的参数解释：

- gradient：有关gradient的介绍见后面（书上的好像不太对，反正看不懂）
- retain_graph：反向传播需要缓存一些中间结果，反向传播之后，这些缓存就被清空，可通过指定这个参数不清空缓存，用来多次反向传播。
- create_graph：构建计算图

In [2]:
a=t.randn(3,4,requires_grad=True)
'''
其他形式：
    a=t.randn(3,4).requires_grad_()

    a=t.randn(3,4)
    a.requires_grad=True
'''
print(a)
b=t.zeros(3,4).requires_grad_()
print(b)

tensor([[-0.2003,  1.4794, -1.1173, -1.0750],
        [ 1.0410,  0.2565,  1.0045, -1.1332],
        [ 0.4874, -1.6257,  0.3305,  0.3567]], requires_grad=True)
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], requires_grad=True)


In [3]:
c=a+b
print(c)
d = c.sum()
print(d)
print(d.backward()) # 反向传播
print(d.requires_grad)

tensor([[-0.2003,  1.4794, -1.1173, -1.0750],
        [ 1.0410,  0.2565,  1.0045, -1.1332],
        [ 0.4874, -1.6257,  0.3305,  0.3567]], grad_fn=<AddBackward0>)
tensor(-0.1954, grad_fn=<SumBackward0>)
None
True


In [4]:
print(a.grad) # 即对a求导

# 此处虽然没有指定c需要求导，但c依赖于a，而a需要求导，
# 因此c的requires_grad属性会自动设为True
print(a.requires_grad, b.requires_grad, c.requires_grad)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
True True True


In [5]:
# 由用户创建的variable属于叶子节点，对应的grad_fn是None
print(a.is_leaf, b.is_leaf, c.is_leaf)

# c.grad是None, 因c不是叶子节点，它的梯度是用来计算a的梯度
# 所以虽然c.requires_grad = True,但其梯度计算完之后即被释放
print(c.grad is None)

True True False
True


上面写了这么多，其实我一个都没看懂。。。😥，感觉讲的怪怪的，一会冒出来一个参数一会儿又冒出来一个参数，这个例子感觉举的也不好，于是从apachecn的pytorch文档又扒了一个例子再来试一遍。
    PyTorch中，所有神经网络的核心是`autograd`包，`autograd`包中的**核心类**是`torch.Tensor`，如果将生成的tensor的属性设置为`.requires_grad=True`，如下面的例子：
 ```python
a=t.randn(3,4,requires_grad=True)
```
那么它将会追踪对于该张量的所有操作,当完成计算后：
- `backward()`方法可以自动计算所有梯度
- 这个张量的所有梯度将会自动累加到`.grad`属性

要阻止一个张量被跟踪历史，可以调用` .detach()`方法将其与计算历史分离，并阻止它未来的计算记录被跟踪

为了防止跟踪历史记录（和使用内存），可以将代码块包装在 with torch.no_grad(): 中。在**评估模型**时特别有用，因为模型可能具有 requires_grad = True 的可训练的参数，但是评估过程并不需要计算梯度。

除了上文提到的`torch.Tensor`类，还有一个类：`Function`对autograd的实现非常重要

- 每个张量都有一个`.grad_fn`属性，该属性引用了创建`Tensor`自身的`Function`（除非这个张量是用户手动创建的，即这个张量的`grad_fn`是`None`）
- `Tensor`和`Function`互相连接生成了一个**无环图(acyclic graph)**

> 在图论中，如果一个有向图从任意顶点出发无法经过若干条边回到该点，则这个图是一个**有向无环图（DAG,directed acyclic graph）**

如果需要计算导数，可以在`Tensor`上调用`.backward()`。如果`Tensor`是一个标量（即它包含一个元素的数据），则不需要为`backward()`指定任何参数，但是如果它有更多的元素，则需要指定一个`gradient`参数，该参数是形状匹配的张量。

In [6]:
x=t.ones(2,2,requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
